In [1]:
import sys
import os
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")
from load_params import load_multigroup_params
from multi_group_dorm_simulation import MultiGroupDormSimulation

In [2]:
dorm_sim = MultiGroupDormSimulation(
            num_dorms=100,
            dorm_population=8000,
            non_dorm_population=26000,
            intra_dorm_contacts=10, # 10 contacts/day within each dorm
            inter_dorm_contacts=10, #10 contacts/day with other people in dorms
            intra_non_dorm_contacts=7.3, # each person in a dorm has this many contacts/day with person not in a dorm
            inter_non_dorm_contacts=3, #contacts/day within non-dorm community
            dorm_test_rate=0.5, # daily test rate within dorms
            non_dorm_test_rate=0.2, #daily test rate in non-dorm
            quarantine_leakage_contacts_dorm=0.3, # dorm -> different dorm contacts/day after dorm is quarantined
            quarantine_leakage_contacts_non_dorm=0.1, # dorm -> non-dorm contacts/day after dorm is quarantined
            quarantine_test_fraction=1.0, # test fraction after dorm goes into quarantine
            dorm_outside_infection_p=0, # outside infection probability for people in dorms
            initial_dorm_prevalence = 0 # initial prevalence (percentage) in the dorms
)

In [3]:
c = dorm_sim.run_new_trajectory(25)

In [4]:
c

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 3, 4, 5, 5, 6, 10, 11, 11, 12]

In [19]:
0 == None

False

In [12]:
dorm_sim.interaction_matrix[200,199]

IndexError: index 200 is out of bounds for axis 0 with size 101

In [13]:
dorm_sim.sims[0].pop_size

80

In [18]:
dorm_sim.interaction_matrix[0,:]

array([10.       ,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010101,
        0.1010101,  0.1010101,  0.1010101,  0.1010101,  0.1010